In [1]:
import numpy as np
import pandas as pd
from os import listdir
import os
import re

In [2]:
os.chdir('../..')

In [3]:
ms_per_frame = 120
data_dir = 'data/gesture/'
is_frame_based = True

In [4]:
ds = pd.DataFrame(columns = ['filename','filetype','movement','person','filenum','frame_length'])

In [5]:
filenames_features = listdir(data_dir + 'features/')
if is_frame_based:
    filenames_labels = listdir(data_dir + 'labels_framebased/')
else:
    filenames_labels = listdir(data_dir + 'labels_timebased/')

In [6]:
re_filename = '(?P<filename>(?P<filetype>[a-z]*)_'
re_movement = '(?P<movement>[a-z]*)_'
re_person = '(?P<person>[a-z]*)_'
re_filenum = '(?P<filenum>\d*)(_'
re_frame_length = '(?P<frame_length>\d*))?\.csv)'

reg = re.compile(re_filename + re_movement + re_person + re_filenum + re_frame_length)

In [7]:
matches = []

for file_name in filenames_features:
    match = reg.search(file_name)
    if match:
        matches.append(match)

for file_name in filenames_labels:
    match = reg.search(file_name)
    if match:
        matches.append(match)

for i, match in enumerate(matches):
    ds.loc[i] = match.groupdict()

ds_features = ds[(ds.filetype == 'features') & (ds.frame_length == '000{0}'.format(str(ms_per_frame))[-3:])]
ds_labels = ds[ds.filetype == 'labels']

comb_ds = pd.merge(ds_features,
                   ds_labels,
                   on = ['movement','person','filenum'],
                   how = 'left',
                   suffixes = ['_features','_labels'])[['movement','filename_features','filename_labels']]

comb_ds = comb_ds.drop(comb_ds[(comb_ds.movement != 'none') & (pd.isnull(comb_ds.filename_labels))].index)
comb_ds = comb_ds.fillna({'filename_labels': 'labels_none.csv'})
comb_ds = comb_ds.reset_index(drop = True)
comb_ds = comb_ds[['filename_features','filename_labels']]

data_source_df = ds
combined_data_files_df = comb_ds

In [8]:
ds_labels.sample(5)

,filename,filetype,movement,person,filenum,frame_length
71,labels_right_c_02.csv,labels,right,c,02,None
70,labels_right_c_01.csv,labels,right,c,01,None
55,labels_land_l_01.csv,labels,land,l,01,None
80,labels_takeoff_l_02.csv,labels,takeoff,l,02,None
73,labels_right_l_01.csv,labels,right,l,01,None


In [9]:
combined_data_files_df.sample(5)

,filename_features,filename_labels
33,features_right_l_03_120.csv,labels_right_l_03.csv
29,features_right_c_02_120.csv,labels_right_c_02.csv
14,features_left_c_02_120.csv,labels_left_c_02.csv
27,features_none_p_03_120.csv,labels_none.csv
39,features_takeoff_l_03_120.csv,labels_takeoff_l_03.csv
